In [3]:
# 필요 라이브러리
import pandas as pd
import numpy as np
import os

!pip install OpenDartReader
import OpenDartReader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# 위 링크에서 발급받은 api 키 입력
my_api = "1b61ac20a39bacd96911f4c99d32245927ea0d8a"
dart = OpenDartReader(my_api)

In [5]:
# 구글 드라이브 설정
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [43]:
# 해당 경로에 있는 .csv 파일명 리스트 가져오기
# import os

############################################ 파일이 저장되어 있는 경로로 설정
path = '/content/gdrive/My Drive/Colab Notebooks/mba-ipo/'
company_file_list = os.listdir(path+'company/')
stock_price_file_list = os.listdir(path+'stock-price/')

company_file_list.sort(reverse=True)
stock_price_file_list.sort(reverse=True)
print(company_file_list)
print(stock_price_file_list)

['신규상장기업현황_2021.csv', '신규상장기업현황_2020.csv', '신규상장기업현황_2019.csv', '신규상장기업현황_2018.csv', '신규상장기업현황_2017.csv', '신규상장기업현황_2016.csv', '신규상장기업현황_2015.csv', '신규상장기업현황_2014.csv', '신규상장기업현황_2013.csv', '신규상장기업현황_2012.csv', '신규상장기업현황_2011.csv']
['종목별공모가대비주가등락률현황_2021.csv', '종목별공모가대비주가등락률현황_2020.csv', '종목별공모가대비주가등락률현황_2019.csv', '종목별공모가대비주가등락률현황_2018.csv', '종목별공모가대비주가등락률현황_2017.csv', '종목별공모가대비주가등락률현황_2016.csv', '종목별공모가대비주가등락률현황_2015.csv', '종목별공모가대비주가등락률현황_2014.csv', '종목별공모가대비주가등락률현황_2013.csv', '종목별공모가대비주가등락률현황_2012.csv', '종목별공모가대비주가등락률현황_2011.csv']


In [44]:
# 특정 기업, 특정 연도의 DART information 추출 함수
def find_financial_indicators(stock_name, year): 
    report = dart.finstate(stock_name, year)
    indicators = ["자산총계", "부채총계", "자본총계", "자본금", "매출액", "영업이익", "당기순이익"]
    if report is None:
        data = [[stock_name, year-1] + [np.nan] * (len(indicators)+1)]
        return pd.DataFrame(data, columns = ["회사명", "연도"] + indicators + ["전기순이익"])
    
    else:
        # 원하는 indicators 만 뽑아옴 (["자산총계", "부채총계", "자본총계", "매출액", "영업이익", "당기순이익"])
        report = report[report['account_nm'].isin(indicators)]
        # print(report)

        # 연결재무제표가 있다면 연결재무제표 사용, 없다면 재무제표 사용
        if sum(report['fs_nm'] == "연결재무제표") >0:
            report = report.loc[report['fs_nm'] == "연결재무제표"]
        
        else:
            report = report.loc[report['fs_nm'] == "재무제표"]
        
        data = []
        record = [stock_name, year]
        for indic in indicators:
            if sum(report['account_nm'] == indic) > 0:
                value = report.loc[report['account_nm'] == indic, 'frmtrm_amount'].iloc[0]
            else:
                value = np.nan
            record.append(value)
        
        if sum(report['account_nm'] == "당기순이익") > 0:
            value = report.loc[report['account_nm'] == "당기순이익", 'bfefrmtrm_amount'].iloc[0]
        else:
            value = np.nan
        record.append(value)

        data.append(record)
        return pd.DataFrame(data,columns = ["회사명", "연도"] + indicators + ["전기순이익"])

In [47]:
# 회사 리스트의 DART information으로 Dataframe 생성 함수
def info_dart(name_list, year_list):
  indicators = ["자산총계", "부채총계", "자본총계", "자본금", "매출액", "영업이익", "당기순이익"]
  finance_df = pd.DataFrame(columns = ["회사명", "연도"] + indicators + ["전기순이익"])

  for name, year in zip(name_list, year_list):
    try:
      finance_df = pd.concat([finance_df, find_financial_indicators(name, int(year))])
    except:
      continue

  finance_df.reset_index(inplace=True, drop=True)
  finance_df = finance_df[finance_df.isnull().sum(axis=1) != 8]
  # print(len(finance_df))

  return finance_df

In [54]:
result = pd.DataFrame(columns=['회사명', '종목코드', '업종', '증권구분', '주관사', '상장일', '공모가 (원)', '(수정)공모가', '상장일 시가', '시가 등락률', '상장일 종가', '종가 등락률', '연도', '자산총계', '부채총계', '자본총계', '매출액', '영업이익', '당기순이익', '전기순이익', 'label'])
for company_file, stock_price_file in zip(company_file_list, stock_price_file_list):
  df_company = pd.read_csv(path+'company/'+company_file)
  df_stock_price = pd.read_csv(path+'stock-price/'+stock_price_file)

  df_company = df_company.iloc[1:].reset_index(drop=True)
  df_stock_price.columns = ['회사명', '주관사', '상장일', '공모가', '(수정)공모가', '상장일 시가', '시가 등락률', '상장일 종가', '종가 등락률', '1개월 주가', '1개월 등락률', '3개월 주가', '3개월 등락률', '6개월 주가', '6개월 등락률', '1년 주가', '1년 등락률', '최근거래일 주가', '최근거래일 등락률']
  df_stock_price = df_stock_price.iloc[1:].reset_index(drop=True)

  df = pd.merge(df_company[['회사명', '종목코드', '상장일', '증권구분', '업종', '공모가 (원)']], df_stock_price[['회사명', '주관사', '(수정)공모가', '상장일 시가', '시가 등락률', '상장일 종가', '종가 등락률']], how='inner', on='회사명')

  # 주관사 -> 주관사 개수 칼럼 변경
  df['주관사'] = df['주관사'].apply(lambda x: len(x.split(',')))

  # dart 정보 가져오기
  name_list = df['회사명']
  year_list = df['상장일'].apply(lambda x: int(x[:4]))

  df_dart = info_dart(name_list, year_list)

  df_target = pd.merge(df, df_dart, how='inner', on='회사명')
  df_target = df_target[['회사명', '종목코드', '업종', '증권구분', '주관사', '상장일', '공모가 (원)', '(수정)공모가', '상장일 시가', '시가 등락률', '상장일 종가', '종가 등락률', '연도', "자산총계", "부채총계", "자본총계", "자본금", "매출액", "영업이익", "당기순이익", "전기순이익"]]
  df_target['label'] = (df_target['(수정)공모가'] < df_target['상장일 종가']).astype(int)

  df_target.to_csv(path+'df'+str(year_list[0])+'.csv', index=False, encoding='utf-8')
  print(str(year_list[0])+': ', len(df_target))
  result = pd.concat([result, df_target], axis=0)

print('Total: ', len(result))
result.to_csv(path+'df_total.csv', index=False, encoding='utf-8')

2021:  78
2020:  53
2019:  57
2018:  62
2017:  48
2016:  41
2015:  41
2014:  0
2013:  0
2012:  0
2011:  0
Total:  380
